## Utils

In [14]:
import math
import math
import torch
import torch.nn as nn
import numpy as np
from skimage.metrics import peak_signal_noise_ratio
import os
import os.path
import numpy as np
import random
import h5py
import cv2
import glob
import torch.utils.data as udata
import torch.optim as optim
import torchvision.utils as utils
from torch.autograd import Variable
from torch.utils.data import DataLoader
from tensorboardX import SummaryWriter

In [15]:
def weights_init_kaiming(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        nn.init.kaiming_normal(m.weight.data, a=0, mode="fan_in")
    elif classname.find("Linear") != -1:
        nn.init.kaiming_normal(m.weight.data, a=0, mode="fan_in")
    elif classname.find("BatchNorm") != -1:
        # nn.init.uniform(m.weight.data, 1.0, 0.02)
        m.weight.data.normal_(mean=0, std=math.sqrt(2.0 / 9.0 / 64.0)).clamp_(
            -0.025, 0.025
        )
        nn.init.constant(m.bias.data, 0.0)

In [16]:
def batch_PSNR(img,imclean,data_range):
    Img = img.data.cpu().numpy().astype(np.float32)
    Iclean = imclean.data.cpu().numpy().astype(np.float32)
    PSNR =0 
    for i in range(Img.shape[0]):
        PSNR += peak_signal_noise_ratio(Iclean[i,:,:,:],Img[i,:,:,:],data_range=data_range)
    return (PSNR/Img.shape[0])


In [17]:
def data_augmentation(image, mode):
    out = np.transpose(image, (1,2,0))
    if mode == 0:
        # original
        out = out
    elif mode == 1:
        # flip up and down
        out = np.flipud(out)
    elif mode == 2:
        # rotate counterwise 90 degree
        out = np.rot90(out)
    elif mode == 3:
        # rotate 90 degree and flip up and down
        out = np.rot90(out)
        out = np.flipud(out)
    elif mode == 4:
        # rotate 180 degree
        out = np.rot90(out, k=2)
    elif mode == 5:
        # rotate 180 degree and flip
        out = np.rot90(out, k=2)
        out = np.flipud(out)
    elif mode == 6:
        # rotate 270 degree
        out = np.rot90(out, k=3)
    elif mode == 7:
        # rotate 270 degree and flip
        out = np.rot90(out, k=3)
        out = np.flipud(out)
    return np.transpose(out, (2,0,1))


## Models

In [18]:
import torch
import torch.nn as nn

In [19]:
class DnCNN(nn.Module):
    def __init__(self, channels, num_of_layers=17, dropout_prob=0.2):
        super(DnCNN, self).__init__()
        kernel_size = 3
        padding = 1
        features = 64
        layers = []

        # First Conv layer without dropout
        layers.append(
            nn.Conv2d(
                in_channels=channels,
                out_channels=features,
                kernel_size=kernel_size,
                padding=padding,
                bias=False,
            )
        )
        layers.append(nn.ReLU(inplace=True))

        # Intermediate layers with dropout
        for _ in range(num_of_layers - 2):
            layers.append(
                nn.Conv2d(
                    in_channels=features,
                    out_channels=features,
                    kernel_size=kernel_size,
                    padding=padding,
                    bias=False,
                )
            )
            layers.append(nn.BatchNorm2d(features))
            layers.append(nn.ReLU(inplace=True))

            # Adding dropout after each ReLU
            layers.append(nn.Dropout(p=dropout_prob))

        # Last Conv layer without dropout
        layers.append(
            nn.Conv2d(
                in_channels=features,
                out_channels=channels,
                kernel_size=kernel_size,
                padding=padding,
                bias=False,
            )
        )

        self.dncnn = nn.Sequential(*layers)

    def forward(self, x):
        out = self.dncnn(x)
        return out

## Dataset

In [20]:
def normalize(data):
    return data/255.0

In [21]:
class Dataset(udata.Dataset):
    def __init__(self, clean_dir, noisy_dir):
        super(Dataset, self).__init__()
        self.clean_files = sorted(
            glob.glob(os.path.join(clean_dir, "*.png"))
        )  # Free noise images
        self.noisy_files = sorted(
            glob.glob(os.path.join(noisy_dir, "*.png"))
        )  # Noisy images
        assert len(self.clean_files) == len(
            self.noisy_files
        ), "Mismatch between clean and noisy images count"
        random.shuffle(
            self.clean_files
        )  # Shuffle the clean files list, which will be used as reference

    def __len__(self):
        return len(self.clean_files)

    def __getitem__(self, index):
        clean_img_path = self.clean_files[index]
        noisy_img_path = self.noisy_files[index]

        # Load clean and noisy images
        clean_img = cv2.imread(
            clean_img_path, cv2.IMREAD_GRAYSCALE
        )  # Load clean image (grayscale)
        noisy_img = cv2.imread(
            noisy_img_path, cv2.IMREAD_GRAYSCALE
        )  # Load noisy image (grayscale)

        # Normalize images
        clean_img = np.float32(normalize(clean_img))
        noisy_img = np.float32(normalize(noisy_img))

        # Convert to PyTorch tensors
        clean_img = torch.Tensor(clean_img).unsqueeze(0)  # Add channel dimension
        noisy_img = torch.Tensor(noisy_img).unsqueeze(0)  # Add channel dimension

        return clean_img, noisy_img

In [22]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
outf= "./logs"
mode = "S"
noiseL = 50
val_noiseL = 25
batchSize = 6
num_of_layers = 17
lr = 1e-3
epochs=5
milestone = 30
clean_dir = "/home/kareem/hacking/research/AI_Love/denoising/DcNN/data/train"
noisy_dir = "/home/kareem/hacking/research/AI_Love/denoising/DcNN/data/noisy_train"
test_clean_dir = "/home/kareem/hacking/research/AI_Love/denoising/DcNN/data/test"
test_noisy_dir = "/home/kareem/hacking/research/AI_Love/denoising/DcNN/data/noisy_test"

In [23]:
# Load dataset
print("Loading dataset ...\n")


dataset_train = Dataset(clean_dir=clean_dir,noisy_dir=noisy_dir)
loader_train = DataLoader(
    dataset=dataset_train, num_workers=4, batch_size=batchSize, shuffle=True
)
print("# of training samples: %d\n" % int(len(dataset_train)))
# Build model
net = DnCNN(channels=1, num_of_layers=num_of_layers,dropout_prob=0.3)
net.apply(weights_init_kaiming)
criterion = nn.MSELoss(size_average=False)
# Move to GPU
device_ids = [0]
model = nn.DataParallel(net, device_ids=device_ids).cuda()
criterion.cuda()
# Optimizer
optimizer = optim.Adam(model.parameters(), lr=lr,weight_decay=1e-4)
# training
writer = SummaryWriter(outf)
step = 0
noiseL_B = [0, 55]  # ingnored when opt.mode=='S'
psnr_val = 0
# training

Loading dataset ...

# of training samples: 260



/tmp/ipykernel_269732/3161319601.py:4: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight.data, a=0, mode="fan_in")
/tmp/ipykernel_269732/3161319601.py:12: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias.data, 0.0)


In [24]:
import torchvision.transforms as transforms
for epoch in range(epochs):
    if epoch < milestone:
        current_lr = lr
    else:
        current_lr = lr / 10.0
    # set learning rate
    for param_group in optimizer.param_groups:
        param_group["lr"] = current_lr
    print("learning rate %f" % current_lr)

    # train
    for i, (clean_img, noisy_img) in enumerate(loader_train, 0):
        # training step
        model.train()
        model.zero_grad()
        optimizer.zero_grad()

        clean_img, noisy_img = Variable(clean_img.cuda()), Variable(noisy_img.cuda())

        out_train = model(noisy_img)
        noise = (
            noisy_img - clean_img
        )  # The difference between noisy and clean is the noise
        loss = criterion(out_train, noise) / (noisy_img.size()[0] * 2)
        loss.backward()
        optimizer.step()

        # results
        model.eval()
        out_train = torch.clamp(noisy_img - model(noisy_img), 0.0, 1.0)
        psnr_train = batch_PSNR(out_train, clean_img, 1.0)
        print(
            "[epoch %d][%d/%d] loss: %.4f PSNR_train: %.4f"
            % (epoch + 1, i + 1, len(loader_train), loss.item(), psnr_train)
        )

        if step % 10 == 0:
            # Log the scalar values
            writer.add_scalar("loss", loss.item(), step)
            writer.add_scalar("PSNR on training data", psnr_train, step)
        step += 1
    # save model
    torch.save(model.state_dict(), os.path.join(outf, "net.pth"))

learning rate 0.001000
[epoch 1][1/44] loss: 1149.3796 PSNR_train: 11.7420


KeyboardInterrupt: 

In [ ]:
dataset_val = Dataset(clean_dir=test_clean_dir, noisy_dir=test_noisy_dir)
loader_val = torch.utils.data.DataLoader(dataset_val, batch_size=1, shuffle=False)
model.eval()
psnr_val = 0
for k in range(len(dataset_val)):
    clean_img, noisy_img = torch.unsqueeze(dataset_val[k][0], 0), torch.unsqueeze(
        dataset_val[k][1], 0
    )

    clean_img, noisy_img = Variable(clean_img.cuda(), volatile=True), Variable(
        noisy_img.cuda(), volatile=True
    )

    out_val = torch.clamp(noisy_img - model(noisy_img), 0.0, 1.0)
    # psnr_val += batch_PSNR(out_val, clean_img, 1.0)
    pnsr_e = batch_PSNR(out_val,clean_img,1.0)
    print("psnre_val",pnsr_e)
    psnr_val += pnsr_e
psnr_val /= len(dataset_val)
print("Final PSNR",psnr_val)

/tmp/ipykernel_269732/3193723546.py:10: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  clean_img, noisy_img = Variable(clean_img.cuda(), volatile=True), Variable(


psnre_val 11.865729423230693
psnre_val 12.802597219245737
psnre_val 13.926203857751673
psnre_val 16.715856226095987
psnre_val 12.926392920142327
psnre_val 9.574606302899324
psnre_val 10.996847642440475
psnre_val 13.353520930891767
psnre_val 11.97394551184109
psnre_val 10.464963214855672
psnre_val 13.035174016090822
psnre_val 13.683507843862499
psnre_val 12.25830511611088
psnre_val 11.099776019629966
psnre_val 12.402361209628479
psnre_val 11.18130648139553
psnre_val 5.34362122434932
psnre_val 12.026570248244628
psnre_val 13.588167894701499
psnre_val 10.780811771280039
psnre_val 11.90709348708676
psnre_val 13.185305840941934
psnre_val 8.98746688537217
psnre_val 11.095027993201573
psnre_val 8.287523943121325
psnre_val 11.40396061409627
psnre_val 12.298201121418469
psnre_val 11.896795591622151
psnre_val 10.041518580716955
psnre_val 11.920560544841239
psnre_val 12.307291770083655
psnre_val 13.204762109610805
psnre_val 10.016509421361059
psnre_val 10.589096062972514
psnre_val 11.280493110834

In [ ]:
#
#! Best value
#Final PSNR 12.945157112665747